# * Interim : 2025 Target Revenue TRUE
    96 Areas

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2025, 3, 3)

## ETL Process...

### Step 1 : Import Data Source

In [12]:
''' Rawdata '''

# Target_Revenue_TRUE_Y2025
src_file = '../../data/interim/Target_Revenue_TRUE_Y2025.xlsx'
src_sheet = 'Rawdata'
src_df = pd.read_excel(src_file, sheet_name=src_sheet, index_col=None) 
# TM_KEY_DAY	METRIC_CD	METRIC_NAME	 METRIC_VALUE 	COMP_CD	VERSION	AREA_CD(TMP)	AREA_DESC(TMP)	AREA_TYPE(TMP)	LOAD_DATE	REMARK	TM_KEY_MTH	AREA_TYPE	AREA_CD	AREA_DESC
src_df_cols = ['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'METRIC_VALUE', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_CD', 'AREA_DESC']
src_df = src_df[src_df_cols]
src_df = src_df.loc[src_df['AREA_TYPE']=="HH"]
src_df.rename(columns={'AREA_CD': 'AREA_KEY', 'METRIC_VALUE': 'MTH_VALUE'}, inplace=True)

# src_df.rename(columns={'AREA_CD': 'AREA_KEY', 'DATA_MONTH': 'MONTH_SHORT', 'DATA_YEAR': 'TM_KEY_YR', 'METRIC_VALUES': 'MTH_VALUE'}, inplace=True)
# src_df['MONTH_SHORT'] = src_df['MONTH_SHORT'].apply(lambda x: x.upper())
# src_df = src_df.replace(np.nan, None)

print(f'\nsrc_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')
src_df.tail(3)


src_df : 768 rows, 9 columns


,TM_KEY_MTH,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC
837,202502,TB4R000100,TVS Revenue,5.977121e+06,True,T,HH,906110,SURAT THANI
838,202502,TB4R000100,TVS Revenue,8.721572e+05,True,T,HH,906111,TRANG
839,202502,TB4R000100,TVS Revenue,3.974766e+05,True,T,HH,906112,YALA


In [25]:
''' Master Data '''

# DIM_TIME
dt_file = '../CFW/data/dim_time.csv'
dt_cols = ['TM_KEY_YR', 'MONTH_SHORT', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH']
dt_df = pd.read_csv(dt_file, usecols=dt_cols)
# dt_df.tail(3)

# DIM_MOOC_AREA
mooc_file = '../CFW/data/dim_mooc_area.csv'
mooc_cols = ['ZONE_TYPE', 'TEAM_CODE', 'ORGID_G', 'TDS_SGMD', 'ORGID_R', 'TDS_RGM_CODE', 'ORGID_H', 'HOP_HINT', 'TDS_PROVINCE', 'PROVINCE_ENG', 'PROVINCE_TH', 'ORGID_HH', 'D_CLUSTER', 'CCAATT', 'REMARK']
mooc_df = pd.read_csv(mooc_file, usecols=mooc_cols)
mooc_df = mooc_df.loc[(mooc_df['REMARK']!='Dummy') & (mooc_df['TEAM_CODE']!='ไม่ระบุ') & (mooc_df['HOP_HINT']!='True Corp')]
# mooc_df.tail(3)

# Create HH level
mooc_hh_df = mooc_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT', 'ORGID_HH', 'D_CLUSTER']].drop_duplicates()
mooc_hh_df.dropna(how='all', inplace=True)
mooc_hh_df['AREA_KEY'] = mooc_hh_df['ORGID_HH'].astype(int).astype(str)
# mooc_hh_df#.tail(3)

In [26]:
''' Example DataFrame '''

# src_df.tail(3)
# dt_df.tail(3)
# mooc_df.tail(3)
# mooc_h_df.tail(3)
# mooc_h_df.loc[mooc_h_df['ORGID_H'].str.contains('^0')].tail(3)

src_df.tail(3)

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC
837,202502,TB4R000100,TVS Revenue,5.977121e+06,True,T,HH,906110,SURAT THANI
838,202502,TB4R000100,TVS Revenue,8.721572e+05,True,T,HH,906111,TRANG
839,202502,TB4R000100,TVS Revenue,3.974766e+05,True,T,HH,906112,YALA


### Step 2 : Aggregate Data

In [34]:
'''
    TB1R000100	Prepaid Revenue : TMH
    TB2R010100	Postpaid Revenue B2C : TMH
    TB3R000100	TOL Revenue
    TB4R000100	TVS Revenue
'''

''' Filter '''
raw_df = src_df
# raw_df = raw_df.loc[raw_df['TM_KEY_MTH'].isin([202501, 202502])]
raw_df = raw_df.loc[raw_df['TM_KEY_MTH']==202501]
# raw_df = raw_df.loc[raw_df['METRIC_CD'].isin(['TB1R000100', 'TB2R010100', 'TB3R000100', 'TB4R000100'])]

''' Data Test '''
# raw_df = raw_df.loc[raw_df['TM_KEY_MTH']==202501]
raw_df = raw_df.loc[raw_df['METRIC_CD']=='TB1R000100']
# raw_df = raw_df.loc[raw_df['AREA_TYPE']=='H']
# raw_df = raw_df.loc[raw_df['AREA_KEY'].isna()]
# raw_df = raw_df.loc[raw_df['AREA_KEY'].isin(['016','040','080'])]
raw_df = raw_df.reset_index(drop=True)

print(f'\nraw_df : {raw_df.shape[0]} rows, {raw_df.shape[1]} columns')
raw_df#.tail(3)


raw_df : 96 rows, 9 columns


,TM_KEY_MTH,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC
0,202501,TB1R000100,Prepaid Revenue : TMH,2.841393e+07,True,T,HH,907030,"BKK : Bang Khen, Lat Phrao, Wang Thonglang"
1,202501,TB1R000100,Prepaid Revenue : TMH,2.222143e+07,True,T,HH,907016,"BKK : Bang Sue, Chatuchak"
2,202501,TB1R000100,Prepaid Revenue : TMH,2.640907e+07,True,T,HH,907017,"BKK : Don Mueang, Sai Mai, Lak Si"
3,202501,TB1R000100,Prepaid Revenue : TMH,3.870089e+07,True,T,HH,907019,"BKK : Lat Krabang, Nong Chok, Khlong Sam Wa"
4,202501,TB1R000100,Prepaid Revenue : TMH,3.027286e+07,True,T,HH,907020,"BKK : Min Buri, Khan Na Yao, Bueng Kum"
...,...,...,...,...,...,...,...,...,...
91,202501,TB1R000100,Prepaid Revenue : TMH,6.485584e+06,True,T,HH,906108,SATUN
92,202501,TB1R000100,Prepaid Revenue : TMH,3.796499e+07,True,T,HH,906109,SONGKHLA
93,202501,TB1R000100,Prepaid Revenue : TMH,3.976183e+07,True,T,HH,906110,SURAT THANI
94,202501,TB1R000100,Prepaid Revenue : TMH,1.441704e+07,True,T,HH,906111,TRANG


In [23]:
''' Join Area '''

merge_corp_df = raw_df.loc[raw_df['AREA_TYPE']=='True corp']
# merge_corp_df.tail(3)

merge_hh_df = pd.merge(raw_df.loc[raw_df['AREA_TYPE']=='HH'], mooc_hh_df, how='left', on='AREA_KEY')
# merge_hh_df.tail(3)

merge_area_df = pd.concat([merge_corp_df, merge_hh_df])
print(f'\nprep_agg_df : {merge_area_df.shape[0]} rows, {merge_area_df.shape[1]} columns')
merge_area_df.tail(3)

# merge_area_df = merge_hh_df
# print(f'\nprep_agg_df : {merge_area_df.shape[0]} rows, {merge_area_df.shape[1]} columns')
# merge_area_df.tail(3)


prep_agg_df : 776 rows, 22 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),...,AREA_KEY,AREA_DESC,CHANNEL_CD,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER
765,20250221,MAR,2025,DB2S000100,Postpaid Gross Adds : DTAC,324.472262,DTAC,T,906110.0,SURAT THANI,...,906110,SURAT THANI,ALL,UPC,GX8,Regional Management 8 (South),84Z,SURAT THANI,906110.0,SURAT THANI
766,20250221,MAR,2025,DB2S000100,Postpaid Gross Adds : DTAC,119.412197,DTAC,T,906111.0,TRANG,...,906111,TRANG,ALL,UPC,GX8,Regional Management 8 (South),93X,"TRANG, SATUN, PHATTHALUNG",906111.0,TRANG
767,20250221,MAR,2025,DB2S000100,Postpaid Gross Adds : DTAC,21.617366,DTAC,T,906112.0,YALA,...,906112,YALA,ALL,UPC,GX8,Regional Management 8 (South),96X,"PATTANI, YALA, NARATHIWAT",906112.0,YALA


In [24]:
''' Join Period '''

merge_period_df = pd.merge(merge_area_df, dt_df, how='left', on=['TM_KEY_YR', 'MONTH_SHORT'])
# merge_period_df.tail(3)

# Prep Aggregate
prep_agg_df = merge_period_df
prep_agg_df['DAY_VALUE'] = prep_agg_df['MTH_VALUE'] / prep_agg_df['DAYS_IN_MONTH']
print(f'\nprep_agg_df : {prep_agg_df.shape[0]} rows, {prep_agg_df.shape[1]} columns')
prep_agg_df.tail(3)


prep_agg_df : 22892 rows, 27 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),...,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,TM_KEY_DAY,DAYS_IN_MONTH,TRUE_TM_KEY_WK,TM_KEY_MTH,DAY_VALUE
22889,20250221,MAR,2025,DB2S000100,Postpaid Gross Adds : DTAC,21.617366,DTAC,T,906112.0,YALA,...,Regional Management 8 (South),96X,"PATTANI, YALA, NARATHIWAT",906112.0,YALA,20250329,31,2025013,202503,0.697334
22890,20250221,MAR,2025,DB2S000100,Postpaid Gross Adds : DTAC,21.617366,DTAC,T,906112.0,YALA,...,Regional Management 8 (South),96X,"PATTANI, YALA, NARATHIWAT",906112.0,YALA,20250330,31,2025013,202503,0.697334
22891,20250221,MAR,2025,DB2S000100,Postpaid Gross Adds : DTAC,21.617366,DTAC,T,906112.0,YALA,...,Regional Management 8 (South),96X,"PATTANI, YALA, NARATHIWAT",906112.0,YALA,20250331,31,2025014,202503,0.697334


In [25]:
''' Aggregate P, G, H, HH '''

agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE'] # , 'FREQUENCY', 'REMARK'

# P : Nationwide
agg_p_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_p_df['AREA_NO'] = 1
agg_p_df['AREA_TYPE'] = 'P'
agg_p_df['AREA_CD'] = 'P'
agg_p_df['AREA_NAME'] = 'Nationwide'
agg_p_df = agg_p_df.loc[:, agg_cols]
# agg_p_df[agg_p_df['TM_KEY_DAY']==20240501]

# G : Region
agg_g_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_G', 'TDS_SGMD']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_g_df['AREA_NO'] = 2
agg_g_df['AREA_TYPE'] = 'G'
agg_g_df.rename(columns={'ORGID_G': 'AREA_CD'}, inplace=True)
agg_g_df.rename(columns={'TDS_SGMD': 'AREA_NAME'}, inplace=True)
agg_g_df = agg_g_df.loc[:, agg_cols]
# agg_g_df[agg_g_df['TM_KEY_DAY']==20240501]

# H : HOP_HINT
agg_h_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_H', 'HOP_HINT']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_h_df['AREA_NO'] = 3
agg_h_df['AREA_TYPE'] = 'H'
agg_h_df.rename(columns={'ORGID_H': 'AREA_CD'}, inplace=True)
agg_h_df.rename(columns={'HOP_HINT': 'AREA_NAME'}, inplace=True)
agg_h_df = agg_h_df.loc[:, agg_cols]
# agg_h_df[agg_h_df['TM_KEY_DAY']==20240501]

# HH : D_CLUSTER
agg_hh_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_HH', 'D_CLUSTER']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_hh_df['AREA_NO'] = 4
agg_hh_df['AREA_TYPE'] = 'HH'
agg_hh_df['ORGID_HH'] = agg_hh_df['ORGID_HH'].astype(int).astype(str)
agg_hh_df.rename(columns={'ORGID_HH': 'AREA_CD'}, inplace=True)
agg_hh_df.rename(columns={'D_CLUSTER': 'AREA_NAME'}, inplace=True)
agg_hh_df = agg_hh_df.loc[:, agg_cols]
# agg_hh_df[agg_hh_df['TM_KEY_DAY']==20240601]

# Concat DataFrame
last_agg_all_df = pd.concat([agg_p_df, agg_g_df, agg_h_df, agg_hh_df], ignore_index=True)
last_agg_all_df['AGG_TYPE'] = 'S'
last_agg_all_df['FREQUENCY'] = 'Daily'
# last_agg_all_df['REMARK'] = last_agg_all_df['TM_KEY_MTH'].apply(lambda x: 'H Level 64 Province' if x>=202401 and x<=202403 else 'HH Level 96 Cluster')
last_agg_all_df['REMARK'] = 'HH Level 96 Cluster'

print(f'\nlast_agg_all_df : {last_agg_all_df.shape[0]} rows, {last_agg_all_df.shape[1]} columns')
# last_agg_all_df.loc[last_agg_all_df['TM_KEY_DAY']==20240601]
last_agg_all_df.tail(3)


last_agg_all_df : 39884 rows, 17 columns


,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,DAY_VALUE,MTH_VALUE,AGG_TYPE,FREQUENCY,REMARK
39881,2025,202503,2025014,20250331,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,4,HH,910096,SURIN,4.531306,140.470482,S,Daily,HH Level 96 Cluster
39882,2025,202503,2025014,20250331,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,4,HH,910097,UBON RATCHATHANI,23.209381,719.490800,S,Daily,HH Level 96 Cluster
39883,2025,202503,2025014,20250331,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,4,HH,910098,YASOTHON,0.000000,0.000000,S,Daily,HH Level 96 Cluster


### Step 3 : Insert to "INTERIM_VINSIGHT_DATA"
    Delete -> Insert

In [26]:
''' Input Parameter '''

# Create list
month_list = last_agg_all_df['TM_KEY_MTH'].drop_duplicates().tolist()
mt_cd_list = last_agg_all_df['METRIC_CD'].drop_duplicates().tolist()

if len(mt_cd_list) == 1:
    mt_cd_list = str(mt_cd_list).replace(r'[', '(').replace(r']', ')')
else:
    mt_cd_list = tuple(mt_cd_list)

# Create Param
# v_param = dict(mth_start=202406, mth_end=202408, metric_cd=mt_cd_list)
v_param = dict(mth_start=min(month_list), mth_end=max(month_list), metric_cd=mt_cd_list)
v_target_schema = 'AUTOKPI'
v_target_table = 'INTERIM_VINSIGHT_DATA'

# query_delete = f"DELETE {v_target_schema}.{v_target_table} WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} AND METRIC_CD IN {v_param['metric_cd']}"
query_delete = f"""
    DELETE {v_target_schema}.{v_target_table} 
    WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} 
    AND METRIC_CD IN {v_param['metric_cd']}
"""

print(f"\nParameter...\n\n   -> TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']}\n   -> METRIC_CD IN {v_param['metric_cd']}")
print(f'\nDataFrame...\n\n   -> last_agg_all_df : {last_agg_all_df.shape[0]} rows, {last_agg_all_df.shape[1]} columns') 
print(f'\nquery_delete...\n{query_delete}')


Parameter...

   -> TM_KEY_MTH BETWEEN 202502 AND 202503
   -> METRIC_CD IN ('DB1R000900', 'DB1S000101', 'DB2R000500', 'DB2S000100')

DataFrame...

   -> last_agg_all_df : 39884 rows, 17 columns

query_delete...

    DELETE AUTOKPI.INTERIM_VINSIGHT_DATA 
    WHERE TM_KEY_MTH BETWEEN 202502 AND 202503 
    AND METRIC_CD IN ('DB1R000900', 'DB1S000101', 'DB2R000500', 'DB2S000100')



In [27]:
''' Load Data '''

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')


# Create rows from DataFrame
rows = [tuple(x) for x in last_agg_all_df.values]


# Connect : AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()
print(f'\nProcessing...')


try:
    # # Truncate
    # cur.execute(f"TRUNCATE TABLE {v_target_schema}.{v_target_table}")
    # print(f'\n   -> TRUNCATE : "{v_target_table}" : Done !')

    # Delete
    cur.execute(query_delete)
    print(f'\n   -> DELETE : "{v_target_table}" : Done !')
    
    # Insert
    cur.executemany(f"""
        INSERT INTO {v_target_table} 
        (TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, VERSION, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, DAY_VALUE, MTH_VALUE, AGG_TYPE, FREQUENCY, REMARK) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17)
        """, rows)
    print(f'\n   -> INSERT : "{v_target_table}" : Done !')

    cur.close()
    conn.commit()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    print(f'\nJob Done !!!')



Job Start... 2025-02-24, 13:44:20

AKPIPRD : Connected

Processing...

   -> DELETE : "INTERIM_VINSIGHT_DATA" : Done !

   -> INSERT : "INTERIM_VINSIGHT_DATA" : Done !

AKPIPRD : Disconnected

Job Done !!!
